In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv_relu(in_ch,out_ch,size=3,rate=1):
  conv_relu = nn.Sequential(nn.Conv2d(in_ch,
                                      out_ch,
                                      kernel_size=size,
                                      stride=1,
                                      padding=rate,
                                      dilation=rate),
                            nn.ReLU())
  return conv_relu

class VGG16(nn.Module):
  def __init__(self):
    super(VGG16,self).__init__()
    self.features1 = nn.Sequential(conv_relu(3,64,3,1),
                                   conv_relu(64,64,3,1),
                                   nn.MaxPool2d(3,stride=2,padding=1)) # 1/2 , 3x3 MaxPool 사용 , padding=1을 추가해줘서 MaxPool2d(2,2,0) 과 output size는 동일 (1/2 시켜주는것)
    self.features2 = nn.Sequential(conv_relu(64,128,3,1),
                                   conv_relu(128,128,3,1),
                                   nn.MaxPool2d(3,stride=2,padding=1)) # 1/2
    self.features3 = nn.Sequential(conv_relu(128,256,3,1),
                                   conv_relu(256,256,3,1),
                                   conv_relu(256, 256, 3, 1),
                                   nn.MaxPool2d(3,stride=2,padding=1)) # 1/2
    self.features4 = nn.Sequential(conv_relu(256, 512, 3, 1),
                                   conv_relu(512, 512, 3, 1),
                                   conv_relu(512, 512, 3, 1),
                                   nn.MaxPool2d(3, stride=1, padding=1)) # 원본이미지 사이즈 그대로 유지
    self.features5 = nn.Sequential(conv_relu(512, 512, 3, rate=2), # dilated rate 2 적용 ,(padding도 2 같이 적용, dilated rate만큼 padding도 같이 적용해줘야지 원본 이미지 사이즈 유지됨)
                                   conv_relu(512, 512, 3, rate=2),
                                   conv_relu(512, 512, 3, rate=2),
                                   nn.MaxPool2d(3, stride=1, padding=1), 
                                   nn.AvgPool2d(3, stride=1, padding=1)) # 마지막 stride=1로 해서 두 layer 크기 유지 
  def forward(self, x):
    out = self.features1(x)
    out = self.features2(out)
    out = self.features3(out)
    out = self.features4(out)
    out = self.features5(out)
    return out

class classifier(nn.Module):
    def __init__(self, num_classes): 
        super(classifier, self).__init__()
        self.classifier = nn.Sequential(conv_relu(512, 1024, 3, rate=12), 
                                       nn.Dropout2d(0.5), 
                                       conv_relu(1024, 1024, 1, 1), 
                                       nn.Dropout2d(0.5), 
                                       nn.Conv2d(1024, num_classes, 1)
                                       )
    def forward(self, x): 
        out = self.classifier(x)
        return out 

class DeepLabV1(nn.Module):
    def __init__(self, backbone, classifier, upsampling=8):
        super(DeepLabV1, self).__init__()
        self.backbone = backbone
        self.classifier = classifier
        self.upsampling = upsampling

    def forward(self, x):
        x = self.backbone(x)
        _, _, feature_map_h, feature_map_w = x.size()
        x = self.classifier(x)
        out = F.interpolate(x, size=(feature_map_h * self.upsampling, feature_map_w * self.upsampling), mode="bilinear")
        return out

In [5]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'

# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test
backbone = VGG16()
classifier = classifier(num_classes=12)
model = DeepLabV1(backbone=backbone, classifier=classifier)

x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])
